In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Importing Required Libraries
import numpy as np
import pandas as pd
from typing import List, Dict
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
%cd /content/drive/MyDrive/ML_FSP
songs = pd.read_csv('songdata.csv')
songs.head()

/content/drive/MyDrive/ML_FSP


,artist,song,link,text
0,ABBA,Ahe's My Kind Of Girl,/a/abba/ahes+my+kind+of+girl_20598417.html,"Look at her face, it's a wonderful face \nAnd..."
1,ABBA,"Andante, Andante",/a/abba/andante+andante_20002708.html,"Take it easy with me, please \nTouch me gentl..."
2,ABBA,As Good As New,/a/abba/as+good+as+new_20003033.html,I'll never know why I had to go \nWhy I had t...
3,ABBA,Bang,/a/abba/bang_20598415.html,Making somebody happy is a question of give an...
4,ABBA,Bang-A-Boomerang,/a/abba/bang+a+boomerang_20002668.html,Making somebody happy is a question of give an...


In [ ]:
#Because of the dataset being so big, we are going to resample only 5000 random songs.
songs = songs.sample(n=5000).drop('link', axis=1).reset_index(drop=True)

In [ ]:
#Replacing /n with ''
songs['text'] = songs['text'].str.replace(r'\n', '')

<ipython-input-5-b00373e343e5>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  songs['text'] = songs['text'].str.replace(r'\n', '')


In [ ]:
tfidf = TfidfVectorizer(analyzer='word', stop_words='english')

In [ ]:
lyrics_matrix = tfidf.fit_transform(songs['text'])

In [ ]:
cosine_similarities = cosine_similarity(lyrics_matrix)

In [ ]:
similarities = {}

In [ ]:
for i in range(len(cosine_similarities)):
    # Now we'll sort each element in cosine_similarities and get the indexes of the songs.
    similar_indices = cosine_similarities[i].argsort()[:-50:-1]
    # Store in similarities each name of the 50 most similar songs.
    similarities[songs['song'].iloc[i]] = [(cosine_similarities[i][x], songs['song'][x], songs['artist'][x]) for x in similar_indices][1:]

In [ ]:
class ContentBasedRecommender:
    def __init__(self, matrix):
        self.matrix_similar = matrix

    def _print_message(self, song, recom_song):
        rec_items = len(recom_song)

        print(f'The {rec_items} recommended songs for {song} are:')
        for i in range(rec_items):
            print(f"Number {i+1}:")
            print(f"{recom_song[i][1]} by {recom_song[i][2]} with {round(recom_song[i][0], 3)} similarity score")
            print("--------------------")

    def recommend(self, recommendation):
        # Get song to find recommendations for
        song = recommendation['song']
        # Get number of songs to recommend
        number_songs = recommendation['number_songs']
        # Get the number of songs most similars from matrix similarities
        recom_song = self.matrix_similar[song][:number_songs]
        # print each item
        self._print_message(song=song, recom_song=recom_song)

In [ ]:
recommendation = ContentBasedRecommender(similarities)

In [ ]:
song_recommendation = {
    "song": songs['song'].iloc[10],
    "number_songs": 4
}

In [ ]:
recommendation.recommend(song_recommendation)

The 4 recommended songs for Just A Kid are:
Number 1:
Let Me Sleep by Pearl Jam with 0.193 similarity score
--------------------
Number 2:
All The King's Horses by Dusty Springfield with 0.192 similarity score
--------------------
Number 3:
Spread The Love by Kenny Chesney with 0.142 similarity score
--------------------
Number 4:
Mary's Little Boy Child by Neil Diamond with 0.14 similarity score
--------------------


In [ ]:
#Testing whole with random Input
song_recommendation2 = {
    "song": songs['song'].iloc[np.random.randint(5000)],
    "number_songs": 4
}
recommendation.recommend(song_recommendation2)

The 4 recommended songs for Let's See Action are:
Number 1:
I'd Have You Anytime by George Harrison with 0.554 similarity score
--------------------
Number 2:
Let It Down by George Harrison with 0.504 similarity score
--------------------
Number 3:
Don't Let Go by Foreigner with 0.467 similarity score
--------------------
Number 4:
Let Me Down Easy by Cher with 0.444 similarity score
--------------------
